# Mental Health Chatbot

## Goal


## Imports

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import datetime
import sys
import os
import wandb
import gradio as gr
import psycopg2
from dotenv import load_dotenv
from datasets import load_dataset, concatenate_datasets
from sentence_transformers import SentenceTransformer
load_dotenv()
NEON_PG_CONNECTION_URL = os.environ['NEON_PG_CONNECTION_URL']

## DB, Wandb Connection & Setup

In [41]:
try:
    connection = psycopg2.connect(NEON_PG_CONNECTION_URL)
    connection.autocommit = True
    print("Connected to Neon Postgres!")
except Exception as e:
    print("Cannot connect to Neon Postgres:", e)

cursor = connection.cursor()

# wandb.init(project="mental_health_chatbot")

Connected to Neon Postgres!


## Data Pre-Processing

In [42]:
dataset1 = load_dataset("Amod/mental_health_counseling_conversations")
dataset2 = load_dataset("mpingale/mental-health-chat-dataset")

# Combine datasets into a single DataFrame
df1 = pd.DataFrame(dataset1['train'])
df2 = pd.DataFrame(dataset2['train'])
# print(df1.head())
# print(df2.head())
df1 = df1.rename(columns={"Context": "Question", "Response": "Answer"})
df2 = df2.rename(columns={"questionText": "Question", "answerText": "Answer"})
df2 = df2.drop(columns=["questionID", "questionTitle", "questionLink", "topic", "therapistInfo", "therapistURL", "upvotes", "views", "text"])
df = pd.concat([df1, df2])
print(df)

# Drop duplicates & NAs
df = df.drop_duplicates(subset=["Question", "Answer"])
df = df.dropna(subset=["Question", "Answer"])

# Load Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")
# Encode questions and answers
questions = df["Question"].tolist()
answers = df["Answer"].tolist()
print(f"questions: {questions[:5]}")
print(f"answers: {answers[:5]}")
question_vectors = model.encode(questions)
answer_vectors = model.encode(answers)
print(f"question_vectors: {question_vectors[:5]}")

                                               Question  \
0     I'm going through some things with my feelings...   
1     I'm going through some things with my feelings...   
2     I'm going through some things with my feelings...   
3     I'm going through some things with my feelings...   
4     I'm going through some things with my feelings...   
...                                                 ...   
2770  What are some difficulties that a counselor ca...   
2771  What are some difficulties that a counselor ca...   
2772  What are some difficulties that a counselor ca...   
2773  What are some difficulties that a counselor ca...   
2774  What are some difficulties that a counselor ca...   

                                                 Answer  
0     If everyone thinks you're worthless, then mayb...  
1     Hello, and thank you for your question and see...  
2     First thing I'd suggest is getting the sleep y...  
3     Therapy is essential for those that are feelin...  
4

/Users/hubert_1/.pyenv/versions/3.11.9/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


questions: ["I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?", "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?", "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless t

## Insert Dataset to NEON

In [43]:
try:
    cursor.execute("""DROP TABLE IF EXISTS mental_health_qa;""")
    print("Dropped table mental_health_qa.")
except Exception as e:
    print("Cannot drop table mental_health_qa:", e)

try:
    cursor.execute(
        """
        CREATE TABLE mental_health_qa (
            id BIGSERIAL PRIMARY KEY,
            question TEXT,
            answer TEXT,
            question_vector VECTOR(384),
            answer_vector VECTOR(384)
        );
    """
    )
    print("Created table mental_heatlh_qa.")
except Exception as e:
    print("Cannot create table mental_health_qa:", e)


Dropped table mental_health_qa.
Created table mental_heatlh_qa.


In [44]:
for i in range(len(df)):
    cursor.execute(
        """
        INSERT INTO mental_health_qa (question, answer, question_vector, answer_vector)
        VALUES (%s, %s, %s, %s);
        """,
        (
            df["Question"].iloc[i], 
            df["Answer"].iloc[i], 
            question_vectors[i].tolist(),  # Convert numpy array to list
            answer_vectors[i].tolist()     # Convert numpy array to list
        )
    )
print("Inserted data into mental_health_qa.")
connection.close()
print("Connection closed.")

Inserted data into mental_health_qa.
Connection closed.
